# 1. Installing libraries 

In [ ]:
# !pip install neuralforecast datasetsforecast

# 2. Load ETTm2 Data

In [ ]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon

In [ ]:
# Change this to your own data to try the model
Y_df, _, _ = LongHorizon.load(directory='./', group='ETTm2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

n_time = len(Y_df.ds.unique())
print(n_time)
val_size = int(.2 * n_time)
test_size = int(.2 * n_time)
val_size=val_size
test_size=test_size
print(val_size,test_size)

57600
11520 11520


# 3. Train models

In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MAE
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer, FEDformer, PatchTST, TSMixer

In [ ]:
horizon  = 15
n_series = len(Y_df.unique_id.unique())
n_block  = 1
ff_dim   = 16
target_slice = None
# print(type(n_series),n_series)

models = [TSMixer(h=horizon,
                input_size=horizon,
                # max_steps=1000,
                # val_check_steps=100,
                # early_stop_patience_steps=3,
                n_series=n_series,
                n_block=n_block,
                ff_dim=ff_dim,
                target_slice = None,
                random_seed=1)
         ]

Seed set to 1


In [ ]:
nf = NeuralForecast(
    models=models,
    freq='15min')

In [ ]:
Y_hat_df = nf.cross_validation(df=Y_df,
                               val_size=val_size,
                               test_size=test_size,
                               n_windows=None)

Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Predicting: |                                                                                                 …

In [ ]:
Y_hat_df.head()

,ds,cutoff,TSMixer,y
unique_id,,,,
HUFL,2017-10-24 00:00:00,2017-10-23 23:45:00,-1.061760,-0.977673
HUFL,2017-10-24 00:15:00,2017-10-23 23:45:00,-1.032367,-0.865620
HUFL,2017-10-24 00:30:00,2017-10-23 23:45:00,-0.693272,-0.961624
HUFL,2017-10-24 00:45:00,2017-10-23 23:45:00,-0.618313,-1.049700
HUFL,2017-10-24 01:00:00,2017-10-23 23:45:00,-0.641085,-0.953600


# 4. Evaluate Results

In [ ]:
import matplotlib.pyplot as plt
Y_hat_df_v1 = Y_hat_df.reset_index(drop=False).drop(columns=['unique_id','ds','cutoff','TSMixer','y'])
Y_hat_df_v1 = Y_hat_df
Y_hat_df_v1['unique_id'] = Y_hat_df_v1.index
# Y_hat_df_v1.head()

In [ ]:
from neuralforecast.losses.numpy import mae,mape
# mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
# mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
# mae_patchtst = mae(Y_hat_df['y'], Y_hat_df['PatchTST'])
mae_tsmixer = mae(Y_hat_df['y'], Y_hat_df['TSMixer'])
mape_tsmixer = mape(Y_hat_df['y'], Y_hat_df['TSMixer'])

# print(f'Informer: {mae_informer:.3f}')
# print(f'Autoformer: {mae_autoformer:.3f}')
# print(f'PatchTST: {mae_patchtst:.3f}')
print(f'TSMixer MAE: {mae_tsmixer:.3f}')
print(f'TSMixer MAPE: {mape_tsmixer:.3f}')

TSMixer MAE: 0.620
TSMixer MAPE: 2.191


In [ ]:
# #| export
# Y_hat_df_v1['ds'] = pd.to_datetime(Y_hat_df_v1['ds'])
# max_date = Y_hat_df_v1['ds'].max()
# print(max_date)

# Y_hat_df_v2 = Y_hat_df_v1[Y_hat_df_v1["ds"] >= "2018-01-01 23:45:50"]
# len(Y_hat_df_v2[Y_hat_df_v2['unique_id']=='OT'] )

In [ ]:
# Y_plot = Y_hat_df_v2[Y_hat_df_v2['unique_id']=='OT'] # OT dataset
# cutoffs = Y_hat_df_v2['cutoff'].unique()[::horizon]
# Y_plot = Y_plot[Y_hat_df_v2['cutoff'].isin(cutoffs)]

# plt.figure(figsize=(20,5))
# plt.plot(Y_plot['ds'], Y_plot['y'], label='True')
# # plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')
# # plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')
# # plt.plot(Y_plot['ds'], Y_plot['PatchTST'], label='PatchTST')
# plt.plot(Y_plot['ds'], Y_plot['TSMixer'], label='TSMixer')
# plt.xlabel('Datestamp')
# plt.ylabel('OT')
# plt.grid()
# plt.legend()